# ANN from scratch

Kerakli kutubxonalar

In [1]:
import numpy as np
from math import *
import random
import json

In [2]:
class MyModel:  
    
    def __init__(self,layers:list , learning_rate):
        self.k = len(layers) -1  # layerlar soni
        self.W = [np.random.random( (layers[l], layers[l+1]) ) - 0.5 for l in range(self.k )]
        self.b = [np.random.random( (layers[l+1],1) ) - 0.5 for l in range(self.k )]
        self.Z = self.k * [0] 
        self.A = self.k * [0] 
        self.dW = self.k * [0] 
        self.db = self.k * [0] 
        self.learning_rate = learning_rate
    def forward(self,X):
        self.Z[0] = X.dot(self.W[0]) + self.b[0].T
        self.A[0] = self.relu(self.Z[0])
        for l in range(1,self.k-1):
            self.Z[l] = self.A[l-1].dot(self.W[l])+ self.b[l].T
            self.A[l] = self.relu(self.Z[l])
        self.Z[self.k-1] = self.A[self.k-2].dot(self.W[self.k-1])+self.b[self.k-1].T
        self.A[self.k-1] = self.sigmoid(self.Z[self.k-1])
        return self.A[self.k-1]
    def loss(self,X,y):
        a = self.forward(X)
        y = y.reshape(-1,1)
        #mean squared error loss
        L = np.sum((y-a)**2)/X.shape[1]
        return L
    def dloss(self,X,y):
        a = self.forward(X)
        y = y.reshape(-1,1)
        delta = (2*(a -y)/(X.shape[1] if X.ndim!=1 else 1) * self.dsigmoid(self.Z[-1])).T
        return delta
    def backward(self,X, y, a):
        y = y.reshape(-1,1)
        if X.ndim != 1:
            m = X.shape[0]
        else:
            m = 1
        delta = self.dloss(X,y) # delta_L
        for l in range(self.k-1, 0,-1):
            self.dW[l] = np.dot(delta, self.A[l-1])
            self.db[l] = np.sum(delta, axis = 1).reshape(1,-1)
            delta = np.dot(self.W[l],delta) * self.dsigmoid(self.Z[l-1]).T
        self.dW[0] = np.dot(delta, X)
        self.db[0] = np.sum(delta,axis = 1).reshape(1,-1)
        return self.dW, self.db
    def train(self,X,y):
        a = self.forward(X)
        dW,db = self.backward(X,y,a)
        for l in range(self.k):
            self.W[l] = self.W[l]- self.learning_rate*dW[l].T
            self.b[l] = self.b[l] - self.learning_rate*db[l].T
    # MBGD optimizer
    def generate_batch(self,X,y, minibatch_size):
        n_rows = X.shape[0]
        batch_index = np.random.choice(np.arange(n_rows), minibatch_size, replace = False)
        return X[batch_index,:], y[batch_index]
    # SGD optimizer
    def Stochastic_gd(self,X,y):
        n_rows = X.shape[0]
        index = np.random.choice(np.arange(n_rows), replace = False)
        return X[index,:], y[index]
    # BGD optimizer
    def Batch_gd(self,X,y):
        return X, y
    # sigmoid funksiyasi
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    # sigmoid funksiyasi hosilasi
    def dsigmoid(self,x):
        return self.sigmoid(x)*(1-self.sigmoid(x))

    # relu funksiyasi
    def relu(self,x):
        return np.maximum(0,x)
    # relu funksiyasini hosilasi
    def drelu(self,x):
        return x > 0

#### Model test

In [11]:
nn = MyModel([3,4,3,1],learning_rate = 0.05)
X_train = np.array([[1,0,0],
                    [0,1,0],
                    [0,0,1],
                    [1,1,0],
                    [1,0,1],
                    [0,1,1],
                    [1,1,1],
                    [0,0,0]])

y_train =  np.array([  1,      0,      0,      1,    1,    0,      1,    0])
X_test = np.array([[1,0,0],
                   [0,0,1]])
y_test = np.array( [ 1, 0 ])
n_epochs = 10000    # epochlar soni
#minibatch_size = 8
for epoch in range(n_epochs):
    X, y = nn.Batch_gd(X_train,y_train)
    nn.train(X,y)
    print("b ->",nn.b[-1].ravel())
    print("w ->",nn.W[-1].ravel())
    print("loss",nn.loss(X,y))
a = nn.forward(X_test)
y_true = y_test
print(f"predicted y: {a.squeeze()}, true y: {y_true.squeeze()}")

b -> [0.21018633]
w -> [ 0.25065861 -0.43434235 -0.03447207]
loss 0.6747551001515761
b -> [0.20698479]
w -> [ 0.25065861 -0.43434235 -0.03328076]
loss 0.6745228133030143
b -> [0.20382585]
w -> [ 0.25065861 -0.43434235 -0.03207043]
loss 0.6742950122559906
b -> [0.20070875]
w -> [ 0.25065861 -0.43434235 -0.03084142]
loss 0.6740715303538831
b -> [0.19763272]
w -> [ 0.25065861 -0.43434235 -0.02959407]
loss 0.6738522066050048
b -> [0.19459702]
w -> [ 0.25065861 -0.43434235 -0.02832869]
loss 0.6736368855003634
b -> [0.19160092]
w -> [ 0.25065861 -0.43434235 -0.02704559]
loss 0.6734254168358184
b -> [0.18864368]
w -> [ 0.25065861 -0.43434235 -0.02574508]
loss 0.6732176555386427
b -> [0.18572461]
w -> [ 0.25065861 -0.43434235 -0.02442748]
loss 0.6730134614984896
b -> [0.18284298]
w -> [ 0.25065861 -0.43434235 -0.02309306]
loss 0.6728126994027485
b -> [0.17999813]
w -> [ 0.25065861 -0.43434235 -0.02174214]
loss 0.6726152385762653
b -> [0.17718935]
w -> [ 0.25065861 -0.43434235 -0.02037499]
loss

b -> [-0.11613274]
w -> [ 0.25065861 -0.46828014  0.4220929 ]
loss 0.6350115859457713
b -> [-0.11731472]
w -> [ 0.25065861 -0.46863241  0.42492655]
loss 0.6347205483886332
b -> [-0.1184986]
w -> [ 0.25065861 -0.46898646  0.42776739]
loss 0.6344274774886175
b -> [-0.11968441]
w -> [ 0.25065861 -0.4693423   0.43061545]
loss 0.6341323557046085
b -> [-0.12087218]
w -> [ 0.25065861 -0.46969994  0.43347078]
loss 0.6338351653972718
b -> [-0.12206194]
w -> [ 0.25065861 -0.47005936  0.4363334 ]
loss 0.6335358888286512
b -> [-0.12325373]
w -> [ 0.25065861 -0.47042057  0.43920335]
loss 0.63323450816178
b -> [-0.12444757]
w -> [ 0.25065861 -0.47078357  0.44208067]
loss 0.6329310054603052
b -> [-0.12564348]
w -> [ 0.25065861 -0.47114835  0.44496537]
loss 0.6326253626881261
b -> [-0.12684151]
w -> [ 0.25065861 -0.47151493  0.44785751]
loss 0.6323175617090442
b -> [-0.12804166]
w -> [ 0.25065861 -0.4718833   0.4507571 ]
loss 0.6320075842864303
b -> [-0.12924399]
w -> [ 0.25065861 -0.47225345  0.45366

loss 0.5234932588291804
b -> [-0.38111084]
w -> [ 0.25065861 -0.57115497  1.09907379]
loss 0.5224229876583469
b -> [-0.38279829]
w -> [ 0.25065861 -0.57201669  1.10375927]
loss 0.5213466193743528
b -> [-0.3844871]
w -> [ 0.25065861 -0.57288174  1.10845612]
loss 0.5202641489205312
b -> [-0.38617724]
w -> [ 0.25065861 -0.5737501   1.1131643 ]
loss 0.5191755718625789
b -> [-0.38786864]
w -> [ 0.25065861 -0.57462174  1.1178838 ]
loss 0.5180808843985388
b -> [-0.38956127]
w -> [ 0.25065861 -0.57549665  1.12261459]
loss 0.5169800833687622
b -> [-0.39125508]
w -> [ 0.25065861 -0.57637478  1.12735663]
loss 0.5158731662658457
b -> [-0.39295002]
w -> [ 0.25065861 -0.57725613  1.13210991]
loss 0.5147601312445358
b -> [-0.39464604]
w -> [ 0.25065861 -0.57814065  1.13687439]
loss 0.5136409771315975
b -> [-0.39634309]
w -> [ 0.25065861 -0.57902832  1.14165003]
loss 0.5125157034356408
b -> [-0.39804113]
w -> [ 0.25065861 -0.57991913  1.14643681]
loss 0.5113843103568979
b -> [-0.3997401]
w -> [ 0.2506

b -> [-0.74548751]
w -> [ 0.25065861 -0.81851803  2.24624548]
loss 0.22760691437507194
b -> [-0.74698408]
w -> [ 0.25065861 -0.81983911  2.25100741]
loss 0.2264880028748173
b -> [-0.74847936]
w -> [ 0.25065861 -0.82116112  2.2557593 ]
loss 0.22539556800411087
b -> [-0.74997216]
w -> [ 0.25065861 -0.82248404  2.26050119]
loss 0.2243106351671893
b -> [-0.75146222]
w -> [ 0.25065861 -0.82380785  2.26523307]
loss 0.2232281277591477
b -> [-0.75294947]
w -> [ 0.25065861 -0.82513253  2.26995494]
loss 0.2221480714975342
b -> [-0.75443388]
w -> [ 0.25065861 -0.82645804  2.27466675]
loss 0.22107049269079096
b -> [-0.75591539]
w -> [ 0.25065861 -0.82778438  2.27936848]
loss 0.21999541819755705
b -> [-0.75739396]
w -> [ 0.25065861 -0.82911151  2.2840601 ]
loss 0.2189228753870097
b -> [-0.75886957]
w -> [ 0.25065861 -0.83043941  2.28874158]
loss 0.2178528921002123
b -> [-0.76034216]
w -> [ 0.25065861 -0.83176806  2.29341289]
loss 0.21678549661244015
b -> [-0.76181171]
w -> [ 0.25065861 -0.83309744 

b -> [-1.06510363]
w -> [ 0.25065861 -1.16700756  3.09326936]
loss 0.07401248502078638
b -> [-1.06609521]
w -> [ 0.25065861 -1.16829395  3.09541137]
loss 0.07370376708087534
b -> [-1.06708442]
w -> [ 0.25065861 -1.16957849  3.0975463 ]
loss 0.07339655002098898
b -> [-1.06807127]
w -> [ 0.25065861 -1.17086117  3.09967418]
loss 0.07309082789753446
b -> [-1.06905574]
w -> [ 0.25065861 -1.17214199  3.10179503]
loss 0.07278659476012898
b -> [-1.07003786]
w -> [ 0.25065861 -1.17342094  3.10390888]
loss 0.07248384465211116
b -> [-1.07101761]
w -> [ 0.25065861 -1.17469801  3.10601575]
loss 0.07218257161104717
b -> [-1.07199501]
w -> [ 0.25065861 -1.17597321  3.10811567]
loss 0.07188276966923289
b -> [-1.07297005]
w -> [ 0.25065861 -1.17724652  3.11020866]
loss 0.0715844328541914
b -> [-1.07394275]
w -> [ 0.25065861 -1.17851794  3.11229476]
loss 0.07128755518916535
b -> [-1.0749131]
w -> [ 0.25065861 -1.17978746  3.11437398]
loss 0.07099213069360504
b -> [-1.07588111]
w -> [ 0.25065861 -1.18105

...

b -> [-1.77052463]
w -> [ 0.25065861 -2.37283591  4.37747181]
loss 0.00048693096664920757
b -> [-1.77053785]
w -> [ 0.25065861 -2.37286504  4.37749618]
loss 0.0004868606706707335
b -> [-1.77055106]
w -> [ 0.25065861 -2.37289417  4.37752055]
loss 0.00048679039358418344
b -> [-1.77056428]
w -> [ 0.25065861 -2.3729233   4.37754492]
loss 0.00048672013538216663
b -> [-1.77057749]
w -> [ 0.25065861 -2.37295242  4.37756928]
loss 0.0004866498960573046
b -> [-1.7705907]
w -> [ 0.25065861 -2.37298153  4.37759364]
loss 0.00048657967560221334
b -> [-1.77060391]
w -> [ 0.25065861 -2.37301065  4.37761799]
loss 0.00048650947400952124
b -> [-1.77061712]
w -> [ 0.25065861 -2.37303976  4.37764235]
loss 0.0004864392912718575
b -> [-1.77063032]
w -> [ 0.25065861 -2.37306886  4.37766669]
loss 0.0004863691273818537
b -> [-1.77064353]
w -> [ 0.25065861 -2.37309796  4.37769104]
loss 0.0004862989823321443
b -> [-1.77065673]
w -> [ 0.25065861 -2.37312706  4.37771538]
loss 0.00048622885611536996
b -> [-1.7706699

b -> [-1.77175901]
w -> [ 0.25065861 -2.37555778  4.3797485 ]
loss 0.0004804047425545852
b -> [-1.77177205]
w -> [ 0.25065861 -2.37558655  4.37977257]
loss 0.0004803361906302742
b -> [-1.77178509]
w -> [ 0.25065861 -2.37561533  4.37979663]
loss 0.0004802676569197761
b -> [-1.77179813]
w -> [ 0.25065861 -2.3756441   4.37982069]
loss 0.0004801991414160479
b -> [-1.77181117]
w -> [ 0.25065861 -2.37567286  4.37984475]
loss 0.00048013064411204956
b -> [-1.77182421]
w -> [ 0.25065861 -2.37570163  4.3798688 ]
loss 0.0004800621650007468
b -> [-1.77183724]
w -> [ 0.25065861 -2.37573039  4.37989285]
loss 0.0004799937040751133
b -> [-1.77185027]
w -> [ 0.25065861 -2.37575914  4.3799169 ]
loss 0.00047992526132811824
b -> [-1.7718633]
w -> [ 0.25065861 -2.37578789  4.37994094]
loss 0.00047985683675273787
b -> [-1.77187633]
w -> [ 0.25065861 -2.37581664  4.37996498]
loss 0.00047978843034195763
b -> [-1.77188936]
w -> [ 0.25065861 -2.37584538  4.37998902]
loss 0.0004797200420887533
b -> [-1.77190239]

loss 0.0004691381689724547
b -> [-1.7739377]
w -> [ 0.25065861 -2.38036894  4.38377066]
loss 0.0004690725808920394
b -> [-1.77395044]
w -> [ 0.25065861 -2.3803971   4.38379419]
loss 0.00046900700989084017
b -> [-1.77396318]
w -> [ 0.25065861 -2.38042526  4.38381773]
loss 0.00046894145596239166
b -> [-1.77397592]
w -> [ 0.25065861 -2.38045341  4.38384125]
loss 0.00046887591910022307
b -> [-1.77398865]
w -> [ 0.25065861 -2.38048156  4.38386478]
loss 0.0004688103992978687
b -> [-1.77400139]
w -> [ 0.25065861 -2.38050971  4.3838883 ]
loss 0.00046874489654886656
b -> [-1.77401412]
w -> [ 0.25065861 -2.38053785  4.38391182]
loss 0.00046867941084675495
b -> [-1.77402685]
w -> [ 0.25065861 -2.38056599  4.38393533]
loss 0.0004686139421850805
b -> [-1.77403958]
w -> [ 0.25065861 -2.38059412  4.38395885]
loss 0.00046854849055739093
b -> [-1.7740523]
w -> [ 0.25065861 -2.38062226  4.38398236]
loss 0.00046848305595723126
b -> [-1.77406503]
w -> [ 0.25065861 -2.38065038  4.38400586]
loss 0.000468417

loss 0.00045628983465682187
b -> [-1.77646373]
w -> [ 0.25065861 -2.38595837  4.38844008]
loss 0.0004562275535635561
b -> [-1.77647613]
w -> [ 0.25065861 -2.38598582  4.38846301]
loss 0.00045616528831061656
b -> [-1.77648852]
w -> [ 0.25065861 -2.38601328  4.38848594]
loss 0.00045610303889213975
b -> [-1.77650091]
w -> [ 0.25065861 -2.38604073  4.38850886]
loss 0.00045604080530226195
b -> [-1.77651331]
w -> [ 0.25065861 -2.38606818  4.38853178]
loss 0.0004559785875351259
b -> [-1.77652569]
w -> [ 0.25065861 -2.38609562  4.3885547 ]
loss 0.00045591638558487865
b -> [-1.77653808]
w -> [ 0.25065861 -2.38612306  4.38857762]
loss 0.00045585419944566675
b -> [-1.77655047]
w -> [ 0.25065861 -2.3861505   4.38860053]
loss 0.00045579202911163644
b -> [-1.77656285]
w -> [ 0.25065861 -2.38617794  4.38862344]
loss 0.00045572987457693847
b -> [-1.77657523]
w -> [ 0.25065861 -2.38620537  4.38864634]
loss 0.0004556677358357357
b -> [-1.77658761]
w -> [ 0.25065861 -2.3862328   4.38866925]
loss 0.000455

b -> [-1.77877731]
w -> [ 0.25065861 -2.39108832  4.39272251]
loss 0.00044473438793137915
b -> [-1.7787894]
w -> [ 0.25065861 -2.39111515  4.3927449 ]
loss 0.00044467501431914944
b -> [-1.77880148]
w -> [ 0.25065861 -2.39114198  4.39276728]
loss 0.00044461565548223754
b -> [-1.77881357]
w -> [ 0.25065861 -2.3911688   4.39278967]
loss 0.0004445563114152875
b -> [-1.77882565]
w -> [ 0.25065861 -2.39119562  4.39281205]
loss 0.00044449698211295125
b -> [-1.77883773]
w -> [ 0.25065861 -2.39122243  4.39283442]
loss 0.00044443766756987996
b -> [-1.77884981]
w -> [ 0.25065861 -2.39124924  4.3928568 ]
loss 0.0004443783677807284
b -> [-1.77886189]
w -> [ 0.25065861 -2.39127605  4.39287917]
loss 0.0004443190827401516
b -> [-1.77887396]
w -> [ 0.25065861 -2.39130286  4.39290154]
loss 0.0004442598124428064
b -> [-1.77888604]
w -> [ 0.25065861 -2.39132966  4.3929239 ]
loss 0.00044420055688336636
b -> [-1.77889811]
w -> [ 0.25065861 -2.39135646  4.39294627]
loss 0.00044414131605648584
b -> [-1.778910

#### Modelni parametrlarini saqlash

In [20]:
parametirs = {}
for i in range(nn.k):
    parametirs[f"Layer_{i+1}"] = {"weight":[item for item in np.resize(nn.W[i],(nn.W[i].size))],
            "bias":[item for item in np.resize(nn.b[i],(np.size(nn.b[i])))]}

####  Model parametrlarini .json fayliga saqlash

In [21]:
import json
with open('model.json', 'w+') as json_file:
    json.dump(parametirs, json_file)